In [29]:
import numpy as np
import pandas as pd
import re

In [2]:
data = 'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'

In [3]:
data


'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'

In [4]:
# split rows by '\n'
rows = data.split("\n")
rows

['Airline Code;DelayTimes;FlightCodes;To_From',
 'Air Canada (!);[21, 40];20015.0;WAterLoo_NEWYork',
 '<Air France> (12);[];;Montreal_TORONTO',
 '(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa',
 '12. Air France;[78, 66];;Ottawa_VANcouvER',
 '""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal',
 '']

In [6]:

# form a 2d array
data_s = []
for row in rows:
    # split columns by ";" 
    data_s.append(row.split(";"))

data_s = data_s[: -1]
data_s

[['Airline Code', 'DelayTimes', 'FlightCodes', 'To_From'],
 ['Air Canada (!)', '[21, 40]', '20015.0', 'WAterLoo_NEWYork'],
 ['<Air France> (12)', '[]', '', 'Montreal_TORONTO'],
 ['(Porter Airways. )', '[60, 22, 87]', '20035.0', 'CALgary_Ottawa'],
 ['12. Air France', '[78, 66]', '', 'Ottawa_VANcouvER'],
 ['""".\\.Lufthansa.\\."""', '[12, 33]', '20055.0', 'london_MONTreal']]

In [20]:
# transfer to a numpy array
data_s = np.array(data_s)
columns = data_s[0]
values = data_s[1:]

# create a dataframe
df = pd.DataFrame(data = values, columns = columns)
df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",,Ottawa_VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055.0,london_MONTreal


## 1. FlightCodes column: Some values are null. Flight Codes are supposed to increase by 10 with each row so 1010 and 1030 will have 1020 in the middle. Fill in these missing numbers and make the column an integer column (instead of a float column).


In [21]:
# convert FlightCodes from str to numerical
df['FlightCodes'] = pd.to_numeric(df['FlightCodes'])

# fill the null value with previous value + 10
df['FlightCodes'] = df['FlightCodes'].fillna(df['FlightCodes'].shift() + 10)

# convert to integer column
df['FlightCodes'] = df['FlightCodes'].astype(int)
df


,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015,WAterLoo_NEWYork
1,<Air France> (12),[],20025,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALgary_Ottawa
3,12. Air France,"[78, 66]",20045,Ottawa_VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055,london_MONTreal


## 2. To_From column: Should be split into two separate columns for better analysis! Split on '_' to create two new columns respectively. Also, the case of the column is not very readable, convert the column into capital case.

In [28]:
# split into 2 separate columns
df[['To', 'From']] = df['To_From'].str.upper().str.split('_', expand=True)

# drop the 'To_From' column
df = df.drop('To_From', axis = 1)
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada (!),"[21, 40]",20015,WATERLOO,NEWYORK
1,<Air France> (12),[],20025,MONTREAL,TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALGARY,OTTAWA
3,12. Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055,LONDON,MONTREAL


## 3. Airline Code column: Clean the Airline Codes to have no punctuation except spaces in the middle. E.g. '(Porter Airways. )' should become 'Porter Airways'.

In [30]:
# Clean str to have no punctuation except spaces in middle
def clean_str(str):
    # remove all punctuation except spaces
    rtv = re.sub(r'[^\w\s]', '', str)
    
    return rtv


In [31]:
# apply the function
df['Airline Code'] = df['Airline Code'].apply(clean_str)
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,WATERLOO,NEWYORK
1,Air France 12,[],20025,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALGARY,OTTAWA
3,12 Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055,LONDON,MONTREAL


In [37]:
# not sure if 12 is punctuation... if it is then

def clean_str2(str):
    # remove all punctuation except spaces
    rtv = re.sub(r'[^a-zA-Z\s]', '', str)
    
    return rtv


In [38]:
# apply the function_2
df['Airline Code'] = df['Airline Code'].apply(clean_str2)
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,WATERLOO,NEWYORK
1,Air France,[],20025,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALGARY,OTTAWA
3,Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055,LONDON,MONTREAL
